In [1]:
import os
from IPython.display import Image as ipy_image
from IPython.display import display

#from utils.demo_util import get_shape_comp_model, get_shape_comp_opt, make_dummy_batch
#from utils.qual_util import load_bert2vqsc_model, get_lang_prob, save_mesh_as_gif
from utils.qual_util import get_lang_prob, save_mesh_as_gif
from utils.util import seed_everything
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
import os
from termcolor import colored, cprint
import torch
import utils.util as util
from tqdm.notebook import tqdm
from datasets.text2shape import  Text2Shape


from datasets.text2shapepp_dataset import  Text2ShapePP


%reload_ext autoreload
%autoreload 2

In [33]:
class Opt():
    print("class")
    
def get_shape_comp_opt(gpu_id=0):
    opt = Opt()

    # args
    gpuid=[gpu_id]
    batch_size=16
    max_dataset_size=64

    name='new_bert_overfit'

    # default args
    opt.serial_batches = False
    opt.nThreads = 4

    # important args
    opt.dataset_mode = 'shapenet_code'
    opt.bert_cfg = "./configs/bert2vqsc.yaml"
    opt.seed = 111
    opt.isTrain = True
    opt.gpu_ids = gpuid
    opt.max_dataset_size=64
    opt.device = 'cuda:%s' % gpuid[0]
    opt.batch_size = batch_size
    opt.max_dataset_size = max_dataset_size
    opt.logs_dir="logs"
    opt.name = name
    opt.lr = 1e-4
    opt.save_latest_freq = 1000
    opt.print_freq = 1
    #utils.util.seed_everything(opt.seed)
    opt.phase = 'test'
    opt.nepochs = 100
    opt.nepochs_decay = 60
    opt.mlp_layers = 3
    opt.mlp_hidden = 1024
    return opt
opt = get_shape_comp_opt()

class


In [35]:


dataset = Text2Shape()
#opt = Opt()

dataset.initialize(opt,isTrain=False)

In [17]:
len(dataset)
# import numpy as np
# k = np.arange(0,156707)

64

In [154]:
# k.shape
# test_indices = np.random.choice(156707, 16000, replace=False)

In [156]:
# np.save('./datasets/text2shape_test_indices', test_indices)
# test_indices.shape

(16000,)

In [167]:
# x = np.ones(156707)
# x[test_indices]=0
# train_indices = np.where(x==1)
# train_indices = train_indices[0]
np.save('./datasets/train_indices', train_indices)
#test_indices

In [18]:
for i in tqdm(range(len(dataset))):
    dataset[i]

  0%|          | 0/64 [00:00<?, ?it/s]

In [30]:
dataset[19]["current_text"]

'and rocking feet'

In [26]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

text1 = "thin legs"
text2 = "round arms"
# tokenized1 = tokenizer(text1,return_tensors='pt',padding=True)
# x = bertmodel(**tokenized1).pooler_output
# x.shape
# tokenized2 = tokenizer(text2,return_tensors='pt',padding=True)
# x2 = bertmodel(**tokenized2).pooler_output
# x2.shape
x1 = model.encode(text1)
x2 = model.encode(text2)
x1.shape
x2.shape
model.eval()



SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [27]:
cos = torch.nn.CosineSimilarity(dim=0)

In [28]:
cos(torch.Tensor(x1),torch.Tensor(x2))

tensor(0.4562)

In [123]:
from models.bert2vq_scmodel_v4 import BERT2VQSCModel

In [124]:
m = BERT2VQSCModel()
m.initialize(opt)

---------- Networks initialized -------------
-----------------------------------------------


In [125]:
import copy
data = copy.deepcopy(dataset[4])
# data2 = dataset[2]
data["z_set_prev"] = data["z_set_prev"].unsqueeze(0)
data["z_set_target"] = data["z_set_target"].unsqueeze(0)
data["current_text"] = [data["current_text"]]


In [126]:
#m.net.eval()
m.set_input(data)
m.forward()
m.outp.shape

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])

In [57]:
m.backward()

> /cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/models/bert2vq_scmodel_v4.py(110)backward()
    108         #import pdb;pdb.set_trace()
    109 
--> 110         loss_nll = self.criterion_nce(self.outp, target)
    111 
    112         self.loss = loss_nll

ipdb> self.outp.shape
torch.Size([1, 512, 8, 8, 8])
ipdb> self.z_target
tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  

ipdb> self.z_target.shape
torch.Size([1, 8, 8, 8, 512])
ipdb> target = rearrange(target, 'bs d1 d2 d3 c -> bs c d1 d2 d3')
ipdb> target.shape
torch.Size([1, 512, 8, 8, 8])
ipdb> self.criterion_nce(self.outp, target)
tensor(6.1907, device='cuda:0', grad_fn=<DivBackward1>)
ipdb> target.shape
torch.Size([1, 512, 8, 8, 8])
ipdb> outp.shape
*** NameError: name 'outp' is not defined
ipdb> self.outp.shape
torch.Size([1, 512, 8, 8, 8])
ipdb> target
tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

   

ipdb> target.shape
torch.Size([1, 512, 8, 8, 8])
ipdb> loss(self.outp ,target)
*** NameError: name 'loss' is not defined
ipdb> self.criterion(self.out,target)
*** AttributeError: 'BERT2VQSCModel' object has no attribute 'criterion'
ipdb> quit


In [10]:
x = dataset[0]["z_set_prev"].shape
y = dataset[3]["z_set_prev"].shape
x

torch.Size([8, 8, 8, 512])

In [11]:
x[x!=0].shape

AttributeError: 'int' object has no attribute 'shape'

In [127]:
from torch.utils.data import DataLoader
train_dl =  DataLoader(
            dataset,
            batch_size=opt.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=0)

In [128]:
options=opt
model = m

def train_one_epoch3(pt_profiler=None,epoch=0):
    global total_steps
    global train_loss
    global valid_loss

    epoch_iter = 0
    for i, data in enumerate(train_dl):
        iter_start_time = time.time()
        #visualizer.reset()
        #import pdb;pdb.set_trace()
        total_steps += options.batch_size
        epoch_iter += options.batch_size
        model.set_input(data)

        model.optimize_parameters(total_steps)

        nBatches_has_trained = total_steps // options.batch_size
        # if total_steps % options.print_freq == 0:

        if nBatches_has_trained % options.print_freq == 0:
            errors = model.get_current_errors()
            #import pdb;pdb.set_trace()
            #print(model.loss,"LOSS")
            t = (time.time() - iter_start_time) / options.batch_size
#             visualizer.print_current_errors(
#                 epoch, epoch_iter, total_steps, errors, t)

        # if (nBatches_has_trained % options.display_freq == 0) or i == 0:
        #     # eval
        #     model.inference(data)
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='train')

        #     #model.set_input(next(test_dg))
        #     test_data = next(test_dg)
        #     model.inference(test_data.unsqueeze(0))
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='test')

#         if total_steps % options.save_latest_freq == 0:
#             cprint('saving the latestmodel (epoch %d, total_steps %d)' %
#                    (epoch, total_steps), 'blue')
#             latest_name = f'epoch-latest'
#             model.save(latest_name)

        if pt_profiler is not None:
            pt_profiler.step()

In [129]:
import time
import numpy as np
cprint('[*] Start training. name: %s' % options.name, 'blue')
total_steps = 0
train_loss = []
valid_loss = []
for epoch in tqdm(range(options.nepochs + options.nepochs_decay)):
    epoch_start_time = time.time()
    # epoch_iter = 0

    # profile
    train_one_epoch3(None,epoch)

#     if epoch % 100000 == 0:
#         cprint('saving the model at the end of epoch %d, iters %d' %
#                (epoch, total_steps), 'blue')
#         latest_name = f'epoch-latest'
#         model.save(latest_name)
#         cur_name = f'epoch-{epoch}'
#         model.save(cur_name)

    cprint(f'[*] End of epoch %d / %d \t Time Taken: %d sec \n%s' %
           (
               epoch, options.nepochs + options.nepochs_decay,
               time.time() - epoch_start_time,
               os.path.abspath(os.path.join(options.logs_dir, options.name))
           ), 'blue', attrs=['bold']
           )
    #import pdb;pdb.set_trace()
    print("Model Loss:", m.loss.item())
    model.update_learning_rate()


[*] Start training. name: new_bert_overfit


  0%|          | 0/160 [00:00<?, ?it/s]

[*] End of epoch 0 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 473.99273681640625
[*] learning rate = 0.0000100
[*] End of epoch 1 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 429.40936279296875
[*] learning rate = 0.0000200
[*] End of epoch 2 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 369.51507568359375
[*] learning rate = 0.0000300
[*] End of epoch 3 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 307.4161376953125
[*] learning rate = 0.0000400
[*] End of epoch 4 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 269.9833679199219
[*] learning rate = 0.0000500
[*] End of epoch 5 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src

[*] End of epoch 45 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.9855504035949707
[*] learning rate = 0.0000466
[*] End of epoch 46 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.8187339305877686
[*] learning rate = 0.0000461
[*] End of epoch 47 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.5767035484313965
[*] learning rate = 0.0000456
[*] End of epoch 48 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.5227317810058594
[*] learning rate = 0.0000452
[*] End of epoch 49 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.310540199279785
[*] learning rate = 0.0000447
[*] End of epoch 50 / 160 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Proj

KeyboardInterrupt: 

In [132]:
x1 = dataset[0]
x2 = dataset[2]

In [133]:
x1.keys()

dict_keys(['current_text', 'z_set_prev', 'z_set_target'])

In [134]:
x1["current_text"] = [x1["current_text"]]
x1["z_set_prev"] = x1["z_set_prev"].unsqueeze(0)
x1["z_set_target"] = x1["z_set_target"].unsqueeze(0)
model.set_input(x1)

In [137]:
model.net.eval()
y = model.forward()

In [138]:
model.outp.shape


torch.Size([1, 8, 8, 8, 512])

In [139]:
from einops import rearrange
k2 = x1["z_set_target"]
#k2.shape
#k = rearrange(k2, 'bs d1 d2 d3 c ->bs c d1 d2 d3')

In [140]:
f = model.outp.cpu()

In [142]:
f.shape
k2.shape

torch.Size([1, 8, 8, 8, 512])

In [144]:
cos = torch.nn.CosineSimilarity(dim=-1)

In [145]:
cos(f,k2)

tensor([[[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 0.9992, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

         [[1.0000, 0.9999, 0.9999, 1.0000, 0.9999, 0.9997, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9999, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
          

In [125]:
z1 = torch.rand((1,8,8,8,512))
z2 = torch.zeros((1,8,8,8,512))*-1/512
outp1 = m.net.forward(["wheels"],z1.cuda())
outp2 = m.net.forward(["okkkasfasf"],z2.cuda())

In [126]:
cos(outp1.flatten(),outp2.flatten())

tensor(0.7964, device='cuda:0', grad_fn=<DivBackward0>)

tensor([[[[[0.0023, 0.0020, 0.0018,  ..., 0.0018, 0.0018, 0.0023],
           [0.0020, 0.0018, 0.0018,  ..., 0.0018, 0.0018, 0.0018],
           [0.0022, 0.0020, 0.0018,  ..., 0.0018, 0.0018, 0.0018],
           ...,
           [0.0022, 0.0018, 0.0018,  ..., 0.0020, 0.0018, 0.0023],
           [0.0025, 0.0018, 0.0018,  ..., 0.0022, 0.0018, 0.0018],
           [0.0018, 0.0022, 0.0018,  ..., 0.0018, 0.0018, 0.0021]],

          [[0.0024, 0.0022, 0.0018,  ..., 0.0019, 0.0018, 0.0019],
           [0.0017, 0.0017, 0.0017,  ..., 0.0017, 0.0017, 0.0019],
           [0.0017, 0.0026, 0.0017,  ..., 0.0017, 0.0017, 0.0017],
           ...,
           [0.0017, 0.0017, 0.0017,  ..., 0.0017, 0.0017, 0.0024],
           [0.0018, 0.0018, 0.0018,  ..., 0.0018, 0.0018, 0.0019],
           [0.0018, 0.0018, 0.0018,  ..., 0.0018, 0.0018, 0.0018]],

          [[0.0018, 0.0018, 0.0018,  ..., 0.0018, 0.0018, 0.0018],
           [0.0018, 0.0017, 0.0017,  ..., 0.0017, 0.0017, 0.0026],
           [0.0017, 0.0019

tensor([[[[[0.0021, 0.0024, 0.0018,  ..., 0.0022, 0.0018, 0.0025],
           [0.0022, 0.0025, 0.0019,  ..., 0.0018, 0.0018, 0.0022],
           [0.0026, 0.0025, 0.0019,  ..., 0.0018, 0.0018, 0.0025],
           ...,
           [0.0019, 0.0023, 0.0018,  ..., 0.0018, 0.0018, 0.0021],
           [0.0020, 0.0020, 0.0021,  ..., 0.0018, 0.0018, 0.0019],
           [0.0020, 0.0025, 0.0019,  ..., 0.0022, 0.0018, 0.0018]],

          [[0.0019, 0.0027, 0.0018,  ..., 0.0019, 0.0021, 0.0026],
           [0.0017, 0.0018, 0.0019,  ..., 0.0017, 0.0020, 0.0030],
           [0.0024, 0.0017, 0.0017,  ..., 0.0017, 0.0018, 0.0026],
           ...,
           [0.0018, 0.0017, 0.0017,  ..., 0.0017, 0.0023, 0.0022],
           [0.0018, 0.0018, 0.0024,  ..., 0.0019, 0.0018, 0.0020],
           [0.0018, 0.0021, 0.0018,  ..., 0.0023, 0.0018, 0.0018]],

          [[0.0018, 0.0022, 0.0018,  ..., 0.0021, 0.0018, 0.0026],
           [0.0019, 0.0017, 0.0017,  ..., 0.0019, 0.0017, 0.0021],
           [0.0017, 0.0017

In [45]:
dataset[5]["previous_text"]

KeyError: 'previous_text'

In [58]:
dataset[18]["current_text"]

'has four decorative holes in it'